In [4]:
# simulating temporal inference with active feedback correction — a hallmark of biological intelligence!

import json
import random

# === CONFIGURATION ===
MODEL_FILE = "htpc_model.json"
TEST_SENTENCE = "the dog ran in the park and the cat slept on the mat"
NOISE_PROBABILITY = 0.0  # Simulate top-down ambiguity (try 0.2 for noise)

# === TEXT PROCESSING ===

def normalize_token(token):
    return token.lower().strip(".,!?;:()[]{}\"'")

def tokenize(sentence):
    return [normalize_token(tok) for tok in sentence.strip().split() if tok]

# === LOAD MODEL ===

def load_model(json_path):
    with open(json_path, "r") as f:
        model = json.load(f)

    token_transitions = model['token_transitions']

    bigram_memory = {
        tuple(key.split("|||")): value
        for key, value in model['bigram_memory'].items()
    }

    phrase_memory = {
        tuple(tuple(pair.split("__")) for pair in key.split("|||")): value
        for key, value in model['phrase_memory'].items()
    }

    return token_transitions, bigram_memory, phrase_memory

# === DYNAMIC REMAPPING FUNCTION ===

def dynamic_feedback_for_context(context_tokens, phrase_memory, noise_probability=0.0):
    expectations = set()
    for phrase in phrase_memory:
        flat = [phrase[0][0]] + [pair[1] for pair in phrase]
        for i in range(len(flat) - 1):
            match_len = i + 1
            if flat[:match_len] == context_tokens[-match_len:]:
                next_token = flat[i + 1]
                if random.random() > noise_probability:
                    expectations.add(next_token)
    return expectations

# === PHRASE MATCHING ===

def match_all_phrases(tokens, phrase_memory):
    matched_phrases = []
    n = len(tokens)
    for phrase in phrase_memory:
        phrase_len = len(phrase) + 1
        for i in range(n - phrase_len + 1):
            test_bigrams = tuple((tokens[j], tokens[j + 1]) for j in range(i, i + phrase_len - 1))
            if test_bigrams == phrase:
                phrase_str = " ".join([tokens[i]] + [tokens[i + k + 1] for k in range(len(phrase))])
                matched_phrases.append(phrase_str)
    return matched_phrases

# === RECOGNITION WITH REMAPPING ===

def recognize_patterns(tokens, token_transitions, bigram_memory, phrase_memory, noise_probability=0.0):
    patterns = []
    current_pattern = []

    for i in range(len(tokens)):
        curr_token = tokens[i]

        if i == 0:
            current_pattern.append(curr_token)
            continue

        prev_token = tokens[i - 1]
        bigram = (prev_token, curr_token)

        is_valid_transition = (
            token_transitions.get(prev_token) == curr_token or
            bigram in bigram_memory
        )

        context = current_pattern[-3:]  # limit context window
        dynamic_expectations = dynamic_feedback_for_context(context, phrase_memory, noise_probability)
        top_down_match = curr_token in dynamic_expectations

        if is_valid_transition and top_down_match:
            current_pattern.append(curr_token)
        else:
            print(f"\n⚠️  Inhibition triggered at token '{curr_token}' (position {i})")
            if not is_valid_transition:
                print(f"   └─ Feedforward mismatch: no transition from '{prev_token}'")
            if not top_down_match:
                print(f"   └─ Top-down mismatch: token '{curr_token}' not in expected context {context}")

            if len(current_pattern) > 1:
                matched_phrases = match_all_phrases(current_pattern, phrase_memory)
                patterns.append((current_pattern.copy(), matched_phrases))
            current_pattern = [curr_token]

    if len(current_pattern) > 1:
        matched_phrases = match_all_phrases(current_pattern, phrase_memory)
        patterns.append((current_pattern, matched_phrases))

    return patterns

# === MAIN EXECUTION ===

if __name__ == "__main__":
    random.seed(42)
    token_transitions, bigram_memory, phrase_memory = load_model(MODEL_FILE)
    tokens = tokenize(TEST_SENTENCE)

    print(f"\n🧠 Simulating HTPC Recognition with Dynamic Feedback Remapping (noise={NOISE_PROBABILITY})")
    patterns = recognize_patterns(tokens, token_transitions, bigram_memory, phrase_memory, NOISE_PROBABILITY)

    print("\n🔍 Recognized Patterns:")
    for idx, (tokens, phrases) in enumerate(patterns, 1):
        print(f"\n  Pattern {idx}:")
        print(f"    Tokens: {' '.join(tokens)}")
        if phrases:
            print("    Phrases Matched:")
            for p in phrases:
                print(f"      • {p}")
        else:
            print("    Phrases Matched: (none)")



🧠 Simulating HTPC Recognition with Dynamic Feedback Remapping (noise=0.0)

⚠️  Inhibition triggered at token 'and' (position 6)
   └─ Feedforward mismatch: no transition from 'park'
   └─ Top-down mismatch: token 'and' not in expected context ['in', 'the', 'park']

⚠️  Inhibition triggered at token 'the' (position 7)
   └─ Feedforward mismatch: no transition from 'and'
   └─ Top-down mismatch: token 'the' not in expected context ['and']

🔍 Recognized Patterns:

  Pattern 1:
    Tokens: the dog ran in the park
    Phrases Matched:
      • the dog ran
      • dog ran in
      • ran in the
      • in the park

  Pattern 2:
    Tokens: the cat slept on the mat
    Phrases Matched:
      • the cat slept
      • cat slept on
      • slept on the
      • on the mat
